In [1]:
import pandas as pd
import numpy as np

# Inicialização dos parametros do Q-Learning

In [2]:
states = [0,1,2,3,4,5,6,7]
actions = [0,1,2,3,4,5,6,7]
goal_state = 7

In [3]:
R = [[-1,80,5,100,-1,-1,-1,-1],
     [30,-1,-1,-1,80,100,-1,-1],
     [100,-1,-1,-1,-1,-1,-1,-1],
     [-1,-1,30,-1,-1,-1,100,-1],
     [-1,-1,5,-1,-1,100,-1,-1],
     [-1,30,-1,-1,-1,-1,80,100],
     [-1,-1,-1,-1,30,-1,-1,100],
     [-1,-1,-1,-1,-1,-1,-1,100]]

R = np.array(R)
pd.DataFrame(R, 
             index=['action_'+str(i) for i in range(R.shape[0])],
             columns=['state_'+str(i) for i in range(R.shape[0])])

,state_0,state_1,state_2,state_3,state_4,state_5,state_6,state_7
action_0,-1,80,5,100,-1,-1,-1,-1
action_1,30,-1,-1,-1,80,100,-1,-1
action_2,100,-1,-1,-1,-1,-1,-1,-1
action_3,-1,-1,30,-1,-1,-1,100,-1
action_4,-1,-1,5,-1,-1,100,-1,-1
action_5,-1,30,-1,-1,-1,-1,80,100
action_6,-1,-1,-1,-1,30,-1,-1,100
action_7,-1,-1,-1,-1,-1,-1,-1,100


In [4]:
Q = np.zeros(len(states) * len(actions)).reshape(len(states), len(actions))
print(pd.DataFrame(Q,index=['action_'+str(i) for i in range(R.shape[0])],
                     columns=['state_'+str(i) for i in range(R.shape[0])],))

          state_0  state_1  state_2  state_3  state_4  state_5  state_6  \
action_0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_1      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_2      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_3      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_4      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_5      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_6      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
action_7      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

          state_7  
action_0      0.0  
action_1      0.0  
action_2      0.0  
action_3      0.0  
action_4      0.0  
action_5      0.0  
action_6      0.0  
action_7      0.0  


# Implementação do Q_Learning

In [6]:
class Q_Learning():
    
    def __init__(self, states, actions, R, goal_state, gamma):
        self.states = states
        self.actions = actions
        self.R = R
        self.goal_state = goal_state
        self.gamma = gamma
        self.Q = self.create_Qmatrix()
    
    def get_Qmatrix(self):
        Qdf = pd.DataFrame(self.Q, 
                           index=['state_'+str(i) for i in range(self.Q.shape[0])],
                           columns=['action_'+str(i) for i in range(self.Q.shape[1])]).astype(int)
        return Qdf
        
    def run_episode(self):
        state = np.random.choice(self.states, size = 1)[0]
        next_state = None
        while next_state != self.goal_state:
            possible_next_states = self.get_possible_next_states_R(state)
            next_state = np.random.choice(possible_next_states, size = 1)[0]
            M = self.Q[next_state, self.get_possible_next_states_R(next_state)].max()
            self.Q[state, next_state] = self.R[state, next_state] + self.gamma * M
            state = next_state
        
    def train_agent(self, num_episodes, print_Q_result_after_episodes_end=False):
        for e in range(num_episodes):
            self.run_episode()
        #Criei um booleano, caso nao queira printar o resultado do aprendizado no final do treinamento
        if(print_Q_result_after_episodes_end):
            print(self.Q)
    
    #função auxiliar: a partir da matriz R, a função retorna quais são os possíveis estado a partir de um estado fixado
    def get_possible_next_states_R(self, state):
        possible_next_states = np.argwhere(self.R[state, :]>=0).reshape(-1,)
        return possible_next_states

    #função auxiliar: a partir da matriz Q, a função retorna quais são os possíveis estado a partir de um estado fixado      
    def get_possible_next_states_Q(self, state):
        possible_next_states = np.argwhere(self.Q[state, :]>=1).reshape(-1,)
        return possible_next_states

    def create_Qmatrix(self):
        Q = np.zeros(len(self.states) * len(self.actions)).reshape(len(self.states), len(self.actions))
        return Q

    def caminho_otimo(self, estado):
        #criei uma lista L indicando qual o caminho sugerido pelo agente (representado pela matriz Q)
        L=[]
        L.append(estado)

        proximo_estado = None
        while proximo_estado != self.goal_state:
          proximo_estado = self.get_possible_next_states_Q(estado).max()

          L.append(proximo_estado)

          estado = proximo_estado
    
        return L

# Criei uma metrica para avaliar os resultados

In [7]:
#Criei uma metrica para avaliar qual seria o aprendizado otimo levando em considerações 2 pontos
#1° - Somando todo aprendizado da matriz Q e pegando o maior valor, ja que o maior valor corresponde a matriz que mais aprendeu.
#2° - Considerando o maior valor, qual deles seria adquirido com o menor números de episodios possível.
#Entao, para saber o aprendizado otimo, a função retorna a interceção do maior aprendizado e o menor número de episódios.

def get_aprendizado_otimo(df_aprendizado):
  max_aprendizado = df_aprendizado['total_aprendizado'].values.max()
  todos_max_aprendizado = df_aprendizado.loc[(df_aprendizado['total_aprendizado'].values == max_aprendizado)]
  aprendizado_otimo = todos_max_aprendizado.loc[todos_max_aprendizado['num_ep'] == todos_max_aprendizado['num_ep'].values.min()]
  return aprendizado_otimo

# Treinamento do modelo

In [8]:
#Aqui irei rodar uma especie de gridsearch por implementação propria para treinar o modelo
num_train = 0
df_aprendizado = pd.DataFrame(columns = ['total_aprendizado', 'num_ep'])

for ep in range(1, 100):
  agente = Q_Learning(
    states = states,
    actions = actions,
    R = R,
    goal_state = goal_state,
    gamma = 0 #Considerando gamma 0, o algoritmo não irá considerar o passo subsequente a cada passado.
  )
  num_train += 1
  print("Treinamento número:", num_train)
  agente.train_agent(num_episodes=ep, print_Q_result_after_episodes_end=True)
  agente.get_Qmatrix()
  df_aprendizado.loc[num_train] = [agente.Q.sum(), ep]
  print()

Treinamento número: 1
[[  0.   0.   0. 100.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [100.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. 100.   0.]
 [  0.   0.   5.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  80.   0.]
 [  0.   0.   0.   0.  30.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]

Treinamento número: 2
[[  0.  80.   5.   0.   0.   0.   0.   0.]
 [ 30.   0.   0.   0.  80. 100.   0.   0.]
 [100.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  30.   0.   0.   0.   0.   0.]
 [  0.   0.   5.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  80. 100.]
 [  0.   0.   0.   0.   0.   0.   0. 100.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]

Treinamento número: 3
[[  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 

Printei a matriz Q ao final de cada treinamento para ver a evolução da mesma.

# Avaliação do modelo


In [9]:
print("Melhor numero de episodio é:", get_aprendizado_otimo(df_aprendizado=df_aprendizado).num_ep.values)
print(df_aprendizado)

Melhor numero de episodio é: [6.]
    total_aprendizado  num_ep
1               715.0     1.0
2               810.0     2.0
3               300.0     3.0
4               960.0     4.0
5              1010.0     5.0
..                ...     ...
95             1170.0    95.0
96             1170.0    96.0
97             1170.0    97.0
98             1170.0    98.0
99             1170.0    99.0

[99 rows x 2 columns]


Como podemos ver, o modelo chegou ao seu aprendizado otimo com somente 6 episodios. Não seria necessario treina-lo mais que isso. Lembrando que como os states são aleatórios, no treinamento do modelo, o episodio otimo pode variar um pouco.

#Mostrando o caminho ótimo



In [14]:
agente = Q_Learning(
    states = states,
    actions = actions,
    R = R,
    goal_state = goal_state,
    gamma = 0 #Considerando gamma 0, o algoritmo não irá considerar o passo subsequente a cada passo.
  )
agente.train_agent(num_episodes=10)

In [15]:
for estado in range(0,7):
  print("Estado Inicial,", estado)
  print("Caminho Otimo,", agente.caminho_otimo(estado))
  print()

print("Matriz Q")
agente.get_Qmatrix()

Estado Inicial, 0
Caminho Otimo, [0, 3, 6, 7]

Estado Inicial, 1
Caminho Otimo, [1, 5, 7]

Estado Inicial, 2
Caminho Otimo, [2, 0, 3, 6, 7]

Estado Inicial, 3
Caminho Otimo, [3, 6, 7]

Estado Inicial, 4
Caminho Otimo, [4, 5, 7]

Estado Inicial, 5
Caminho Otimo, [5, 7]

Estado Inicial, 6
Caminho Otimo, [6, 7]

Matriz Q


,action_0,action_1,action_2,action_3,action_4,action_5,action_6,action_7
state_0,0,80,0,100,0,0,0,0
state_1,30,0,0,0,80,100,0,0
state_2,100,0,0,0,0,0,0,0
state_3,0,0,30,0,0,0,100,0
state_4,0,0,5,0,0,100,0,0
state_5,0,30,0,0,0,0,80,100
state_6,0,0,0,0,30,0,0,100
state_7,0,0,0,0,0,0,0,100


De fato o algoritmo está funcionando perfeitamente bem. 
Acima, printei o caminho otimo para todos os estados tomando cada um como inicial.

Para avaliar, vamos analisar a matriz de aprendizado(Q). Obs: Caminhos de valor 0 sao caminhos não possíveis.

Vamos levar em consideração o caminho 0: Os caminhos possíveis de 0 seriam 1, 2 e 3, sendo 3 com a maior recompensa(100). 

Agora, vamos analisar o caminho 3:  Os caminhos possíveis de 3 seriam 2 e 6, sendo 6 com a maior recompensa(100). 

Agora, vamos analisar o caminho 6:  Os caminhos possíveis de 6 seriam 4 e 7, sendo 7 com a maior recompensa(100). 

Sendo assim, fica provado que para o caminho inicial 0 o melhor percurso seria [0, 3, 6, 7].

# Testando varios gammas

Obs: Fiz uns pequenos ajustes afim de otimizar o processo de treino

In [16]:
#Criando esse dataframe(df_result) para verificar posteriormente se existe mais de um caminho possível
df_result = pd.DataFrame(columns = ['Gamma', 'Estado_inicial', 'Caminho_otimo'])

for gamma in np.arange(0.0, 1.1, 0.1):
  num_train = 0
  #df que será usado para verificar o caminho otimo
  df_aprendizado = pd.DataFrame(columns = ['total_aprendizado', 'num_ep'])

  #Treinando de 1 a 200 episodios 
  for ep in range(1, 201):
    agente = Q_Learning(
    states = states,
    actions = actions,
    R = R,
    goal_state = goal_state,
    gamma = gamma
    )
    num_train += 1
    agente.train_agent(num_episodes=ep)
    agente.get_Qmatrix()
    df_aprendizado.loc[num_train] = [agente.Q.sum(), ep]

  agente = Q_Learning(
    states = states,
    actions = actions,
    R = R,
    goal_state = goal_state,
    gamma = gamma
    )
  
  #Treinando o modelo com o episódio ótimo.
  agente.train_agent(num_episodes=int(get_aprendizado_otimo(df_aprendizado=df_aprendizado).num_ep.values[0]))

  #Descobrindo o caminho ótimo de todos os estados
  for estado in range(0,7):
    print("Estado Inicial,", estado)
    print("Caminho Otimo,", agente.caminho_otimo(estado))
    print()
    
    #Populando o dataframe com todos os melhores caminhos possíveis
    df_result.loc[len(df_result)] = [gamma, estado, agente.caminho_otimo(estado)]
  
  print("gamma =", gamma)
  print("Matriz Q")
  print(agente.get_Qmatrix())
  print()
  print('*********')
  print()

Estado Inicial, 0
Caminho Otimo, [0, 3, 6, 7]

Estado Inicial, 1
Caminho Otimo, [1, 5, 7]

Estado Inicial, 2
Caminho Otimo, [2, 0, 3, 6, 7]

Estado Inicial, 3
Caminho Otimo, [3, 6, 7]

Estado Inicial, 4
Caminho Otimo, [4, 5, 7]

Estado Inicial, 5
Caminho Otimo, [5, 7]

Estado Inicial, 6
Caminho Otimo, [6, 7]

gamma = 0.0
Matriz Q
         action_0  action_1  action_2  action_3  action_4  action_5  action_6  \
state_0         0        80         5       100         0         0         0   
state_1        30         0         0         0        80       100         0   
state_2       100         0         0         0         0         0         0   
state_3         0         0        30         0         0         0       100   
state_4         0         0         5         0         0       100         0   
state_5         0        30         0         0         0         0        80   
state_6         0         0         0         0        30         0         0   
state_7         0   

In [17]:
df_result

,Gamma,Estado_inicial,Caminho_otimo
0,0.0,0,"[0, 3, 6, 7]"
1,0.0,1,"[1, 5, 7]"
2,0.0,2,"[2, 0, 3, 6, 7]"
3,0.0,3,"[3, 6, 7]"
4,0.0,4,"[4, 5, 7]"
...,...,...,...
72,1.0,2,"[2, 0, 3, 6, 7]"
73,1.0,3,"[3, 6, 7]"
74,1.0,4,"[4, 5, 7]"
75,1.0,5,"[5, 7]"


#Resultado

In [27]:
print(np.unique(df_result['Caminho_otimo']).reshape(-1,1))

[[list([0, 3, 6, 7])]
 [list([1, 5, 7])]
 [list([2, 0, 3, 6, 7])]
 [list([3, 6, 7])]
 [list([4, 5, 7])]
 [list([5, 7])]
 [list([6, 7])]]


Variando o gamma de 0 a 1, vemos que os caminhos ótimos para cada estado sendo o estado inicial são os mesmos. Ou seja, variar o gamma não faz diferente. Curioso que mesmo considerando o gamma diferente de 0, que anula a visao do ação posterior a cada passo, não faz a minima diferença.
 

#Considerações finais.

Fiz algumas mudanças na classe Q_Learning, afim de englobar melhor o conceito de encapsulamento(de POO), passando a função get_possible_next_states e caminho_otimo(melhor_caminho, como pedido no exercicio) para dentro da classe. 

O caminho otimo se manteve o mesmo independentemente do valor de gamma.

Criei uma metrica para conseguir avaliar o aprendizado ótimo no treinamento do modelo afim de identificar qual seria o episódio otimo para o treinamento do mesmo, pois percebi que a partir de um certo número de episódios o modelo nao evolui mais.

Qualquer dúvida, estou a disposição.
Obrigado.

Kaio.